## IBM Data Science Capstone (Coursera)
### The Battle of Neighborhoods - Opening a new shopping mall in Hyderabad, India.

#### The aim of this project is to select the best location to open a new shopping mall in the city of Hyderabad.

#### - Sourabh Koul

-----
<H3>First, let's import the libraries.

In [3]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files
!pip install geocoder
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
!pip install folium
import folium # map rendering library

print("Libraries imported.")

     |████████████████████████████████| 102kB 7.2MB/s ta 0:00:011
     |████████████████████████████████| 102kB 7.4MB/s ta 0:00:011
Libraries imported.



<H3>Now, let's extract the data from the webpage and convert it into a dataframe.

In [4]:
#Get the data
data = requests.get("https://en.wikipedia.org/wiki/Category:Neighbourhoods_in_Hyderabad,_India").text

#Convert into a beautiful soup object
soup = BeautifulSoup(data, 'html.parser')

In [5]:
#Create an empty list
Neighborhoods = []

In [6]:
# append the data into the list
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    Neighborhoods.append(row.text)

In [7]:
# create a new DataFrame from the list
hyd_df = pd.DataFrame({"Neighborhood": Neighborhoods})

#prints the first 5 neighborhoods
hyd_df.head()

,Neighborhood
0,A. S. Rao Nagar
1,A.C. Guards
2,Abhyudaya Nagar
3,Abids
4,Adikmet


In [8]:
hyd_df.shape
print("There are " + str(hyd_df.shape[0]) + " neighborhoods in the city of Hyderabad." )

There are 200 neighborhoods in the city of Hyderabad.


<H3> Now, let's get the coordinates for each of the neighborhoods.

In [9]:
#Define a function to get coordinates
def get_coord(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Hyderabad, India'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [11]:
# call the function to get the coordinates, store in a new list using list comprehension
coordinates = [ get_coord(neighborhood) for neighborhood in hyd_df["Neighborhood"].tolist() ]
coordinates

[[17.411200000000065, 78.50824000000006],
 [17.392977027745946, 78.45686724051741],
 [17.337650000000053, 78.56414000000007],
 [17.389800000000037, 78.47658000000007],
 [17.410610000000077, 78.51513000000006],
 [17.37751000000003, 78.48005000000006],
 [17.387364823969637, 78.4669870622138],
 [17.34259000000003, 78.47626000000008],
 [17.36068000000006, 78.47998000000007],
 [17.503370000000075, 78.41602000000006],
 [17.535430000000076, 78.54427000000004],
 [17.385820000000024, 78.51836000000003],
 [17.435350000000028, 78.44861000000003],
 [17.40784000000002, 78.49150000000003],
 [17.385140000000035, 78.44738000000007],
 [17.369170000000054, 78.43683000000004],
 [17.40710000000007, 78.50233000000003],
 [17.372720000000072, 78.49047000000007],
 [17.38897000000003, 78.48681000000005],
 [17.39931000000007, 78.49964000000006],
 [17.339920000000063, 78.54553000000004],
 [17.448510000000056, 78.44924000000003],
 [17.415350000000046, 78.43435000000005],
 [17.38859199570786, 78.47665099785392],
 

In [12]:
#Populate the coordinates into a dataframe
df_loc = pd.DataFrame(coordinates, columns = ['Latitude','Longitude'])

In [13]:
#Add the cooordinates to the actual dataframe
hyd_df['Latitude']  = df_loc['Latitude']
hyd_df['Longitude'] = df_loc['Longitude']

In [14]:
hyd_df.head()

,Neighborhood,Latitude,Longitude
0,A. S. Rao Nagar,17.411200,78.508240
1,A.C. Guards,17.392977,78.456867
2,Abhyudaya Nagar,17.337650,78.564140
3,Abids,17.389800,78.476580
4,Adikmet,17.410610,78.515130


<H3>Let's create a map of Hyderabad with other neighborhoods superimposed on it.


In [17]:
# get the coordinates of Hyderabad
address = 'Hyderabad, India'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Hyderabad, India are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Hyderabad, India are 17.38878595, 78.46106473453146.


In [21]:
# create map of Hyderabad using latitude and longitude values
map_hyd = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(hyd_df['Latitude'], hyd_df['Longitude'], hyd_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_hyd)  
    
map_hyd

In [22]:
# save the map as HTML file
map_hyd.save('map_hyd.html')

In [23]:
# save the DataFrame as CSV file
hyd_df.to_csv("hyd_df.csv", index=False)

<H3> Let's use the Foursquare API to explore more.

In [25]:
# @hidden_cell
#Define credentials
CLIENT_ID = 'BA1JO4BDQQCCJMIFXUOGOR4OFHGYSOZNVSKT31U1GP32LSZ1' # your Foursquare ID
CLIENT_SECRET = 'Q2NGS3GAZRCAKBF3AV5V2EQDLQSWQO3WBYHV4O1VPCTLRCPY' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [31]:
radius = 1500
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(hyd_df['Latitude'], hyd_df['Longitude'], hyd_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

<H4> List of top 100 venues within 1500 meters of the localities
    

In [32]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head(10)

(5039, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,A. S. Rao Nagar,17.4112,78.50824,Bawarchi,17.406369,78.497662,Indian Restaurant
1,A. S. Rao Nagar,17.4112,78.50824,Sudharshan Theatre 35mm,17.406530,78.495150,Movie Theater
2,A. S. Rao Nagar,17.4112,78.50824,Subway,17.404173,78.514950,Sandwich Place
3,A. S. Rao Nagar,17.4112,78.50824,Devi 70 MM,17.406329,78.495409,Movie Theater
4,A. S. Rao Nagar,17.4112,78.50824,Baskin-Robbins,17.404311,78.510034,Ice Cream Shop
5,A. S. Rao Nagar,17.4112,78.50824,Spencer's,17.412592,78.498400,Convenience Store
6,A. S. Rao Nagar,17.4112,78.50824,Cafe Coffee Day,17.423051,78.501520,Coffee Shop
7,A. S. Rao Nagar,17.4112,78.50824,Daily Bread,17.403554,78.514961,Café
8,A. S. Rao Nagar,17.4112,78.50824,Crystal Restaurant,17.406608,78.496268,Asian Restaurant
9,A. S. Rao Nagar,17.4112,78.50824,Astoria Restaurant,17.406530,78.497136,Indian Restaurant


<H4> Let's check the number of venues per each neighborhood

In [33]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
A. S. Rao Nagar,25,25,25,25,25,25
A.C. Guards,24,24,24,24,24,24
Abhyudaya Nagar,5,5,5,5,5,5
Abids,47,47,47,47,47,47
Adikmet,16,16,16,16,16,16
Afzal Gunj,31,31,31,31,31,31
Aghapura,38,38,38,38,38,38
"Aliabad, Hyderabad",5,5,5,5,5,5
Alijah Kotla,19,19,19,19,19,19


<H6> Now, we may not have 100 venues in each neighborhood given our limiting radius of 1500 meters. But, let's proceed anyway as we have a lot of data.

<H4> Let's find the unique categories from the venue types.

In [34]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 185 uniques categories.


In [35]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:50]

array(['Indian Restaurant', 'Movie Theater', 'Sandwich Place',
       'Ice Cream Shop', 'Convenience Store', 'Coffee Shop', 'Café',
       'Asian Restaurant', 'Clothing Store', 'Gym', 'Electronics Store',
       'Bakery', 'Light Rail Station', 'Hyderabadi Restaurant', 'Park',
       'Snack Place', 'South Indian Restaurant',
       'Middle Eastern Restaurant', 'Hotel Bar', 'Fried Chicken Joint',
       'Fruit & Vegetable Store', 'Restaurant', 'Food', 'Pizza Place',
       'Mattress Store', 'Juice Bar', 'Hotel', 'Shoe Store',
       'Neighborhood', 'Food Truck', 'Diner', 'Chaat Place',
       'Dessert Shop', 'Stadium', 'Department Store', 'Mobile Phone Shop',
       'Shopping Mall', 'Smoke Shop', 'Bar', 'Fast Food Restaurant',
       'Multiplex', 'Chinese Restaurant', 'Indie Movie Theater',
       'Farmers Market', 'Bookstore', 'Breakfast Spot', 'History Museum',
       'Food Court', 'Bus Station', 'Gift Shop'], dtype=object)

In [37]:
# Check if the result contains "Shopping Mall"
"Shopping Mall" in venues_df['VenueCategory'].unique()

True

<H3> Time to analyze each neighborhood

In [38]:
# one hot encoding
hyd_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
hyd_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [hyd_onehot.columns[-1]] + list(hyd_onehot.columns[:-1])
hyd_onehot = hyd_onehot[fixed_columns]

print(hyd_onehot.shape)
hyd_onehot.head()

(5039, 186)


,Neighborhoods,ATM,Afghan Restaurant,American Restaurant,Arcade,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,BBQ Joint,Bakery,Bank,Bar,Baseball Field,Beer Garden,Bengali Restaurant,Big Box Store,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Buffet,Burger Joint,Bus Station,Business Service,Butcher,Cable Car,Cafeteria,Café,Campground,Candy Store,Chaat Place,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Convention Center,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Donut Shop,Dumpling Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Historic Site,History Museum,Hockey Arena,Hookah Bar,Hotel,Hotel Bar,Hotel Pool,Hunan Restaurant,Hyderabadi Restaurant,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Intersection,Irani Cafe,Irish Pub,Italian Restaurant,Jewelry Store,Juice Bar,Lake,Laser Tag,Light Rail Station,Liquor Store,Lounge,Market,Martial Arts Dojo,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Monument / Landmark,Motel,Motorcycle Shop,Movie Theater,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Night Market,Nightclub,North Indian Restaurant,Optical Shop,Outdoors & Recreation,Park,Parsi Restaurant,Pastry Shop,Performing Arts Venue,Pharmacy,Pizza Place,Platform,Plaza,Pool,Pub,Rajasthani Restaurant,Recreation Center,Residential Building (Apartment / Condo),Resort,Restaurant,River,Sandwich Place,Scenic Lookout,Science Museum,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Social Club,South Indian Restaurant,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Stationery Store,Steakhouse,Supermarket,Tea Room,Tech Startup,Temple,Tex-Mex Restaurant,Thai Restaurant,Theater,Trail,Train Station,Vegetarian / Vegan Restaurant,Wings Joint,Women's Store,Zoo
0,A. S. Rao Nagar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,A. S. Rao Nagar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,A. S. Rao Nagar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,A. S. Rao Nagar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,

<H4> Group by neighborhoods taking mean frequency of occurence of each category

In [39]:
hyd_grouped = hyd_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(hyd_grouped.shape)
hyd_grouped

(200, 186)


,Neighborhoods,ATM,Afghan Restaurant,American Restaurant,Arcade,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,BBQ Joint,Bakery,Bank,Bar,Baseball Field,Beer Garden,Bengali Restaurant,Big Box Store,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Buffet,Burger Joint,Bus Station,Business Service,Butcher,Cable Car,Cafeteria,Café,Campground,Candy Store,Chaat Place,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Convention Center,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Donut Shop,Dumpling Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Historic Site,History Museum,Hockey Arena,Hookah Bar,Hotel,Hotel Bar,Hotel Pool,Hunan Restaurant,Hyderabadi Restaurant,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Intersection,Irani Cafe,Irish Pub,Italian Restaurant,Jewelry Store,Juice Bar,Lake,Laser Tag,Light Rail Station,Liquor Store,Lounge,Market,Martial Arts Dojo,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Monument / Landmark,Motel,Motorcycle Shop,Movie Theater,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Night Market,Nightclub,North Indian Restaurant,Optical Shop,Outdoors & Recreation,Park,Parsi Restaurant,Pastry Shop,Performing Arts Venue,Pharmacy,Pizza Place,Platform,Plaza,Pool,Pub,Rajasthani Restaurant,Recreation Center,Residential Building (Apartment / Condo),Resort,Restaurant,River,Sandwich Place,Scenic Lookout,Science Museum,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Social Club,South Indian Restaurant,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Stationery Store,Steakhouse,Supermarket,Tea Room,Tech Startup,Temple,Tex-Mex Restaurant,Thai Restaurant,Theater,Trail,Train Station,Vegetarian / Vegan Restaurant,Wings Joint,Women's Store,Zoo
0,A. S. Rao Nagar,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,0.000000,0.000000,0.000000,0.040000,0.000000,0.000000,0.00,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.080000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.040000,0.000000,0.080000,0.0,0.000000,0.000000,0.040000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.080000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.080000,0.000000,0.280000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.120000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00

<H4> Now, let's extract the rows containing only Shopping Mall data.

In [41]:
hyd_mall = hyd_grouped[["Neighborhoods","Shopping Mall"]]
hyd_mall.shape
hyd_mall.head()

,Neighborhoods,Shopping Mall
0,A. S. Rao Nagar,0.000000
1,A.C. Guards,0.000000
2,Abhyudaya Nagar,0.000000
3,Abids,0.021277
4,Adikmet,0.000000


<H3> Clustering the neighborhoods

In [52]:
# set number of clusters
kclusters = 3

hyd_clustering = hyd_mall.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(hyd_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:50]


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0], dtype=int32)

In [53]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
hyd_merged = hyd_mall.copy()

# add clustering labels
hyd_merged["Cluster Labels"] = kmeans.labels_

In [54]:
hyd_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
hyd_merged.head(20)

,Neighborhood,Shopping Mall,Cluster Labels
0,A. S. Rao Nagar,0.000000,0
1,A.C. Guards,0.000000,0
2,Abhyudaya Nagar,0.000000,0
3,Abids,0.021277,0
4,Adikmet,0.000000,0
5,Afzal Gunj,0.032258,0
6,Aghapura,0.026316,0
7,"Aliabad, Hyderabad",0.000000,0
8,Alijah Kotla,0.000000,0
9,Allwyn Colony,0.000000,0


In [55]:
# merge hyd_grouped with hyd_data to add latitude/longitude for each neighborhood
hyd_merged = hyd_merged.join(hyd_df.set_index("Neighborhood"), on="Neighborhood")

print(hyd_merged.shape)
hyd_merged.head() # check the last columns!

(200, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,A. S. Rao Nagar,0.000000,0,17.411200,78.508240
1,A.C. Guards,0.000000,0,17.392977,78.456867
2,Abhyudaya Nagar,0.000000,0,17.337650,78.564140
3,Abids,0.021277,0,17.389800,78.476580
4,Adikmet,0.000000,0,17.410610,78.515130


In [56]:
# sort the results by Cluster Labels
print(hyd_merged.shape)
hyd_merged.sort_values(["Cluster Labels"], inplace=True)
hyd_merged

(200, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,A. S. Rao Nagar,0.000000,0,17.411200,78.508240
125,"Mehboob ki Mehendi, Hyderabad",0.000000,0,17.362015,78.470796
126,Mehdipatnam,0.000000,0,17.392630,78.442190
127,Mettuguda,0.000000,0,17.427740,78.528920
128,"Minister Road, Hyderabad",0.000000,0,17.432743,78.484542
129,Mir Alam Tank,0.000000,0,17.355109,78.454123
131,Moazzam Jahi Market,0.023256,0,17.384480,78.474420
132,Moghalpura,0.000000,0,17.358680,78.477120
133,Moosapet,0.000000,0,17.467050,78.428580
134,Moosarambagh,0.000000,0,17.370770,78.518180


<H3> Now, let's visualize the clusters

In [57]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(hyd_merged['Latitude'], hyd_merged['Longitude'], hyd_merged['Neighborhood'], hyd_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [58]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

<H3> Let's examine each cluster

<H4> Cluster 0

In [61]:
hyd_merged.loc[hyd_merged['Cluster Labels'] == 0]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,A. S. Rao Nagar,0.000000,0,17.411200,78.508240
125,"Mehboob ki Mehendi, Hyderabad",0.000000,0,17.362015,78.470796
126,Mehdipatnam,0.000000,0,17.392630,78.442190
127,Mettuguda,0.000000,0,17.427740,78.528920
128,"Minister Road, Hyderabad",0.000000,0,17.432743,78.484542
129,Mir Alam Tank,0.000000,0,17.355109,78.454123
131,Moazzam Jahi Market,0.023256,0,17.384480,78.474420
132,Moghalpura,0.000000,0,17.358680,78.477120
133,Moosapet,0.000000,0,17.467050,78.428580
134,Moosarambagh,0.000000,0,17.370770,78.518180


In [68]:
hyd_merged.loc[hyd_merged['Cluster Labels'] == 0].shape[0]

186

<H4> Cluster 1

In [63]:
hyd_merged.loc[hyd_merged['Cluster Labels'] == 1]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
77,Jeedimetla,0.142857,1,17.521830,78.454330
96,Kukatpally,0.200000,1,17.487350,78.420870
93,"Kothapet, Hyderabad",0.153846,1,17.368830,78.542290
56,Ferozguda,0.111111,1,17.474120,78.426387
107,Lingojiguda,0.125000,1,17.350670,78.534040
85,Karwan,0.100000,1,17.379070,78.436680
84,Karmanghat,0.250000,1,17.340610,78.532580
82,Kapra,0.076923,1,17.478378,78.563662
15,Attapur,0.142857,1,17.369170,78.436830
81,"Kamala Nagar, Hyderabad",0.071429,1,17.365610,78.533050


In [69]:
hyd_merged.loc[hyd_merged['Cluster Labels'] == 1].shape[0]

13

<H4> Cluster 2

In [66]:
hyd_merged.loc[hyd_merged['Cluster Labels'] == 2]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
130,Miyapur,0.5,2,17.42102,78.58244


In [70]:
hyd_merged.loc[hyd_merged['Cluster Labels'] == 2].shape[0]

1

<H3> Observations:

As we can see from the maps and the table data, most of the shopping malls are present in Cluster 0. The remaining clusters i.e. Cluster 1 and Cluster 2 have very few malls. These clusters present a huge opportunity for opening new malls as there will be very little to no competition in trying to open a new shopping mall and generating revenue.  

<H4>Thus, this project recommends property developers and investors to consider the areas in Cluster 1 and Clusters 2 for opening a new shopping mall in Hyderabad.

----
<H4> Limitations:

The analysis done in this project depends on the accuracy of details provided by Foursquare API (Venue Details) and Python Geocoder Package (Coordinates).

----
<H2> Thank You. Hope, you find it useful!